In [16]:
import os
from pathlib import Path
from datetime import datetime, timedelta
from io import StringIO

import numpy as np
import pandas as pd
import requests

from dateutil.tz import tzutc, tzlocal
import math

# Pull Data from NOAA and USGS (stream height and tides)

## Pull the last 24 hours of stream height data from Waikane USGS station

This is the function for requesting USGS data

In [18]:
def USGS_Data_Request(site, start_date, end_date, parameter):
    """
    Download USGS instantaneous data from NWIS API.
    Handles errors gracefully, converts to metric units, and adjusts for HST.
    """
    
    param_units = {
        "00045": "in",       # Precipitation
        "00060": "ft3_s",    # Discharge
        "00065": "ft"        # Gage height
    }

    if parameter not in param_units:
        raise ValueError("Unsupported parameter code. Use '00045', '00060', or '00065'.")

    unit_col = param_units[parameter]

    url = (
        f"https://waterservices.usgs.gov/nwis/iv/?format=json"
        f"&sites={site}&startDT={start_date}&endDT={end_date}"
        f"&parameterCd={parameter}&siteStatus=all"
    )

    try:
        response = requests.get(url, timeout=60)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from USGS: {e}")
        return pd.DataFrame()

    # Handle case where no timeSeries are returned
    if not data.get('value', {}).get('timeSeries'):
        print(f"No data available for site {site} and parameter {parameter}")
        return pd.DataFrame()

    # Parse valid entries
    rows = []
    for ts in data['value']['timeSeries']:
        for entry in ts['values'][0]['value']:
            val_str = entry.get("value", "")
            try:
                val = float(val_str)
            except ValueError:
                val = None

            rows.append({
                "DateTime": entry["dateTime"],
                unit_col: val
            })

    # Build DataFrame
    df = pd.DataFrame(rows)
    df['DateTime'] = pd.to_datetime(df['DateTime']).dt.tz_localize(None)
    df = df[['DateTime', unit_col]]

    return df

Using the function, I will pull the last 24 hours of stream height data

In [19]:
#Waikane

site = '16294900'
parameter = "00065"

# Set start and end dates in UTC with time included
start_date = (datetime.utcnow() - timedelta(days=3)).strftime('%Y-%m-%dT%H:%M')
end_date = datetime.utcnow().strftime('%Y-%m-%dT%H:%M')

df_16294900_H = USGS_Data_Request(site, start_date, end_date, parameter)

C:\Users\bgorb\AppData\Local\Temp\ipykernel_44228\3018553234.py:7: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start_date = (datetime.utcnow() - timedelta(days=3)).strftime('%Y-%m-%dT%H:%M')
C:\Users\bgorb\AppData\Local\Temp\ipykernel_44228\3018553234.py:8: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end_date = datetime.utcnow().strftime('%Y-%m-%dT%H:%M')


In [20]:
#Waiahole

site = '16294100'
parameter = "00065"

# Set start and end dates in UTC with time included
start_date = (datetime.utcnow() - timedelta(days=3)).strftime('%Y-%m-%dT%H:%M')
end_date = datetime.utcnow().strftime('%Y-%m-%dT%H:%M')

df_16294100_H = USGS_Data_Request(site, start_date, end_date, parameter)

C:\Users\bgorb\AppData\Local\Temp\ipykernel_44228\2092448235.py:7: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start_date = (datetime.utcnow() - timedelta(days=3)).strftime('%Y-%m-%dT%H:%M')
C:\Users\bgorb\AppData\Local\Temp\ipykernel_44228\2092448235.py:8: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end_date = datetime.utcnow().strftime('%Y-%m-%dT%H:%M')


In [21]:
# Ensure DateTime is sorted
df_16294100_H = df_16294100_H.sort_values('DateTime').reset_index(drop=True)

# Get the max datetime
max_dt_16294100 = df_16294100_H['DateTime'].max()
print("Max datetime:", max_dt_16294100)

# Compute the cutoff datetime (24 hours before max)
cutoff_dt_16294100 = max_dt_16294100 - pd.Timedelta(hours=24)
print("24 hours earlier:", cutoff_dt_16294100)

# Find the first row where datetime is greater than or equal to cutoff
earlier_index_16294100 = df_16294100_H[df_16294100_H['DateTime'] >= cutoff_dt_16294100].index[0]

Max datetime: 2025-08-07 10:35:00
24 hours earlier: 2025-08-06 10:35:00


In [22]:
# Ensure DateTime is sorted
df_16294900_H = df_16294900_H.sort_values('DateTime').reset_index(drop=True)

# Get the max datetime
max_dt_16294900 = df_16294900_H['DateTime'].max()
print("Max datetime:", max_dt_16294900)

# Compute the cutoff datetime (24 hours before max)
cutoff_dt_16294900 = max_dt_16294900 - pd.Timedelta(hours=24)
print("24 hours earlier:", cutoff_dt_16294900)

# Find the first row where datetime is greater than or equal to cutoff
earlier_index_16294900 = df_16294900_H[df_16294900_H['DateTime'] >= cutoff_dt_16294900].index[0]
#print("Index at least 24 hours before max:", earlier_index_16294900)

Max datetime: 2025-08-07 10:25:00
24 hours earlier: 2025-08-06 10:25:00


## Pull the predicted tides for today from the nearest API NOAA station (Mokuoloe)

In [23]:
def Pull_NOAA_API(station_id, end_date, start_date, datum):
    # Station and API settings
    units = "metric"
    time_zone = "lst"
    clock = "24hour"
    interval = "hilo"
    product = "predictions"
    output_file = station_id + "_tide_predictions.csv"
    
    # Time range: last 3 days

    
    # URL and parameters
    base_url = "https://api.tidesandcurrents.noaa.gov/api/prod/datagetter"
    params = {
        "begin_date": start_date.strftime("%Y%m%d"),
        "end_date": end_date.strftime("%Y%m%d"),
        "station": station_id,
        "product": product,
        "datum": datum,
        "units": units,
        "time_zone": time_zone,
        "clock": clock,
        "interval": interval,
        "format": "csv"
    }
    
    # Download and save
    print(f"Downloading: {params['begin_date']} to {params['end_date']}")
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        if "No tide data was found" in response.text:
            print("❌ No tide data found for this range.")
        else:
            df_tides = pd.read_csv(StringIO(response.text))
            df_tides.to_csv(output_file, index=False)
            print(f"\n✅ Tide data saved to {output_file} ({len(df_tides)} rows)")
    else:
        print(f"❌ Request failed (status code {response.status_code})")
        print(response.text)

    return df_tides


In [24]:
# Time range: last 3 days
end_date = datetime.today() + timedelta(days=2)
start_date = end_date - timedelta(days=5)
station_id = "1612480"

df_Mokouloe = Pull_NOAA_API(station_id, end_date, start_date, datum = 'MLLW')

Downloading: 20250804 to 20250809

✅ Tide data saved to 1612480_tide_predictions.csv (19 rows)


### Convert the predicted tides for Mokouloe to tides for Waikane

In [25]:
def convert_Mokouloe_to_Waikane(Source_df, HEIGHT_OFFSET_HIGH, HEIGHT_OFFSET_LOW, TIME_OFFSET_HIGH, TIME_OFFSET_LOW):
    """
    Adjust tide predictions from Moku o Lo‘e to Waikāne using NOAA time and height offsets.
    Converts predictions from meters to feet and renames column to 'Prediction_ft_MSL'.

    Parameters:
        Source_df (pd.DataFrame): Input tide prediction DataFrame with 'Date Time', 'Prediction', and 'Type'.
        HEIGHT_OFFSET_HIGH (float): Height multiplier for high tides.
        HEIGHT_OFFSET_LOW (float): Height multiplier for low tides.
        TIME_OFFSET_HIGH (int): Time offset in minutes for high tides.
        TIME_OFFSET_LOW (int): Time offset in minutes for low tides.

    Returns:
        pd.DataFrame: Adjusted DataFrame with feet predictions and updated column name.
    """

    required = ['Date Time', 'Prediction', 'Type']
    df = Source_df.copy()
    df.columns = df.columns.str.strip()  # clean column names

    if not all(col in df.columns for col in required):
        raise ValueError(f"Missing required columns. Found columns: {df.columns.tolist()}")

    df['Date Time'] = pd.to_datetime(df['Date Time'])

    def apply_offsets(row):
        if row['Type'] == 'H':
            row['Date Time'] += pd.Timedelta(minutes=TIME_OFFSET_HIGH)
            row['Prediction'] *= HEIGHT_OFFSET_HIGH
        elif row['Type'] == 'L':
            row['Date Time'] += pd.Timedelta(minutes=TIME_OFFSET_LOW)
            row['Prediction'] *= HEIGHT_OFFSET_LOW
        return row

    # Apply NOAA time/height offsets
    df = df.apply(apply_offsets, axis=1)

    # Convert meters to feet and rename column
    df['Prediction_ft_MSL'] = df['Prediction'] * 3.28084
    return df[['Date Time', 'Prediction_ft_MSL', 'Type']]


In [26]:
df_Waikane_HILO = convert_Mokouloe_to_Waikane(df_Mokouloe, HEIGHT_OFFSET_HIGH = 1.13, HEIGHT_OFFSET_LOW = 1, TIME_OFFSET_HIGH = -22, TIME_OFFSET_LOW = -4)

# Calculate the Next Tide from the current time

In [27]:
df_Waikane_HILO["Date Time"] = pd.to_datetime(df_Waikane_HILO["Date Time"])

# Your reference date
ref_date = datetime.now() #current time
next_row = df_Waikane_HILO[df_Waikane_HILO["Date Time"] > ref_date].sort_values("Date Time").reset_index(drop=True)

# Clean column names
next_tide_type = next_row["Type"][0]
next_tide_height = next_row["Prediction_ft_MSL"][0]
next_tide_time = next_row["Date Time"][0]

print(str(next_tide_time) + " " + next_tide_type + " " + str(round(next_tide_height, 2)) + " ft")

2025-08-07 14:16:00 H 2.9 ft


### Create a time series using the HIGH and LOW and convert to feet

In [28]:
def convert_HILO_to_5min_series(df):
    df = df.copy()
    
    # Convert to datetime and round to nearest 5 min
    df["Date Time"] = pd.to_datetime(df["Date Time"])
    df["Date Time"] = df["Date Time"].dt.round("5min")
    df = df.sort_values("Date Time").reset_index(drop=True)

    # Function to generate sine interpolation between two tide points
    def fit_sine_wave(t0, t1, y0, y1):
        duration = (t1 - t0).total_seconds()
        if duration <= 0:
            return [], []

        steps = int(duration / 300) + 1  # 5-minute intervals
        times = [t0 + timedelta(seconds=i * 300) for i in range(steps)]
        elapsed = np.array([(t - t0).total_seconds() for t in times])
        omega = np.pi / duration  # 0 to pi for half-wave
        amplitude = (y1 - y0) / 2
        offset = (y1 + y0) / 2
        levels = amplitude * np.sin(omega * elapsed - np.pi/2) + offset
        return times, levels

    # Build full 5-minute interpolated time series
    full_times = []
    full_levels = []

    for i in range(len(df) - 1):
        t0 = df.loc[i, "Date Time"]
        t1 = df.loc[i + 1, "Date Time"]
        y0 = df.loc[i, "Prediction_ft_MSL"]
        y1 = df.loc[i + 1, "Prediction_ft_MSL"]
        seg_times, seg_levels = fit_sine_wave(t0, t1, y0, y1)
        full_times.extend(seg_times)
        full_levels.extend(seg_levels)

    # Final DataFrame
    df_5min = pd.DataFrame({
        "Datetime": full_times,
        "Predicted_ft_MSL": [round(lvl, 2) for lvl in full_levels]
    }).drop_duplicates("Datetime").sort_values("Datetime").reset_index(drop=True)

    return df_5min


In [29]:
df_Waikane_5min = convert_HILO_to_5min_series(df_Waikane_HILO)

# FOR ANNE:  PLOT A GRAPH SHOWING NEXT DAY, TODAY AND PREVIOUS DAY

# Visualize the Data


## Pull in the flooding threshold values for stream and tide

In [30]:
# Define relative path to the folder
folder_path = os.path.join('.', 'Flooding_Thresholds')

# Define full file paths
stream16294900_thresholds = os.path.join(folder_path, 'Waikane_16294900_stream_flood_thresholds.csv')
stream16294100_thresholds = os.path.join(folder_path, 'Waiahole_16294100_stream_flood_thresholds.csv')

tide_thresholds = os.path.join(folder_path, 'Mokuoloe-Waikane_tide_flood_thresholds.csv')

# Read CSVs
df_stream16294900_thresholds = pd.read_csv(stream16294900_thresholds)
df_stream16294100_thresholds = pd.read_csv(stream16294100_thresholds)
df_tide_thresholds = pd.read_csv(tide_thresholds)


# Rain Gauge Data Stuff (POTENTIAL FOR FLOODING)

In [31]:
#Waiahole Rain Gage
site = '212855157504501'
parameter = "00045"

# Set start and end dates in UTC with time included
start_date = (datetime.utcnow() - timedelta(days=1)).strftime('%Y-%m-%dT%H:%M')
end_date = datetime.utcnow().strftime('%Y-%m-%dT%H:%M')

df_212855157504501_RF = USGS_Data_Request(site, start_date, end_date, parameter)

C:\Users\bgorb\AppData\Local\Temp\ipykernel_44228\270114968.py:6: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start_date = (datetime.utcnow() - timedelta(days=1)).strftime('%Y-%m-%dT%H:%M')
C:\Users\bgorb\AppData\Local\Temp\ipykernel_44228\270114968.py:7: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end_date = datetime.utcnow().strftime('%Y-%m-%dT%H:%M')


In [32]:
# Get the latest timestamp in the DataFrame
latest_time = df_212855157504501_RF["DateTime"].max()
# Define 1-hour window
one_hour_ago = latest_time - timedelta(hours=1)
# Filter for the last 1 hour of data
last_hour_data = df_212855157504501_RF[
    (df_212855157504501_RF["DateTime"] > one_hour_ago) &
    (df_212855157504501_RF["DateTime"] <= latest_time)
]
# Compute the sum
last_hour_sum = last_hour_data["in"].sum()
print(f"Rainfall in the last hour: {last_hour_sum:.2f} inches")

Waiahole_RF_thresholds = os.path.join(folder_path, 'Rain_Gauge_Thresholds_Waiahole.csv')
df_RF_thresholds = pd.read_csv(Waiahole_RF_thresholds)
df_RF_thresholds

Rainfall in the last hour: 0.00 inches


,RF_threshold,Precipitation_in,color
0,min,0.0,green
1,2_year,2.8,yellow
2,10_year,4.1,red
3,max,7.6,red


# Determine if stream is rising or falling !

In [33]:
def get_stream_height_trend(stream_height_dataframe):

    # Ensure datetime is parsed and data is sorted
    stream_height_dataframe['DateTime'] = pd.to_datetime(stream_height_dataframe['DateTime'])
    stream_height_dataframe = stream_height_dataframe.sort_values('DateTime')
    
    # Define the time of the latest reading
    latest_time = stream_height_dataframe['DateTime'].max()
    latest_ft = stream_height_dataframe.loc[stream_height_dataframe['DateTime'] == latest_time, 'ft'].values[0]
    
    # Define the target time (1 hour prior)
    target_time = latest_time - pd.Timedelta(minutes=30)
    
    # Find the closest time BEFORE or NEAR the target (within ±10 minutes)
    time_window = pd.Timedelta(minutes=10)
    possible_matches = stream_height_dataframe[
        (stream_height_dataframe['DateTime'] >= target_time - time_window) &
        (stream_height_dataframe['DateTime'] <= target_time + time_window)
    ]
    
    # Choose the row with the smallest time difference to the target
    threshold_feet = 0.15 #this is the threshold incase the trend is relatively stable
    
    if not possible_matches.empty:
        closest_idx = (possible_matches['DateTime'] - target_time).abs().idxmin()
        prior_ft = stream_height_dataframe.loc[closest_idx, 'ft']
        prior_time = stream_height_dataframe.loc[closest_idx, 'DateTime']
    
        # Compare with threshold
        diff = latest_ft - prior_ft
        if abs(diff) <= threshold_feet:
            trend = "Stable"
        elif diff > 0:
            trend = "Rising"
        else:
            trend = "Falling"
    
    else:
        trend = "Undetermined"
        print("Lack of readings, trend undetermined.")

    return trend

In [34]:
Waikane_trend = get_stream_height_trend(stream_height_dataframe = df_16294900_H)
Waiahole_trend = get_stream_height_trend(stream_height_dataframe = df_16294100_H)